In [1]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv("/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story", header=None, names=["text"])

In [13]:
df.shape

(24, 1)

In [15]:
df['text'][0]

'(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?'